<a href="https://colab.research.google.com/github/syedmuneeb321/LangGraph-Explorations/blob/main/Getting_Started_With_LangGraph/agent_with_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai tavily-python langchain_community

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

gemini_api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain_community.tools import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
# make tool and bind llm
tool = TavilySearchResults(max_results=2)
tool_with_llm = llm.bind_tools([tool])

In [ ]:
# state making

class State(TypedDict):
    messages: Annotated[list,add_messages]


In [ ]:
# make chat bot
def chat_bot(state:State)->State:
  response = tool_with_llm.invoke(state["messages"])
  return {"messages":[response]}

In [ ]:
# make a tool call node
from langgraph.prebuilt import ToolNode,tools_condition

tool_call_node = ToolNode([tool])

ModuleNotFoundError: No module named 'langgraph'

In [ ]:
#

graph_builder = StateGraph(State)

# add node
graph_builder.add_node("chat_bot",chat_bot)
graph_builder.add_node("tools",tool_call_node)

# logic

graph_builder.add_edge(START,"chat_bot")
graph_builder.add_conditional_edges("chat_bot",tools_condition)
graph_builder.add_edge("tools","chat_bot")


# create a memory instance and give compoile graph check pointer parameter
memory = MemorySaver()
app = graph_builder.compile(checkpointer=memory)


NameError: name 'StateGraph' is not defined

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:

input = {"messages":[{"role":"user","content":"what is "}]}

thread_config = {"configurable":{"thread_id":"1"}}

for event in app.stream(input=input,config=thread_config,stream_mode="values"):
  # print(event)
  event['messages'][-1].pretty_print()

================================ Human Message =================================

what is langchain
================================== Ai Message ==================================

LangChain is a framework for developing applications powered by large language models (LLMs). It allows you to connect LLMs to various data sources and tools, enabling them to perform complex tasks like:

* **Question Answering:** Retrieve information from databases, websites, or documents and answer questions based on the retrieved data.
* **Summarization:** Summarize long documents or articles.
* **Text Generation:** Generate creative content, write different kinds of text formats (emails, articles, code), and translate languages.
* **Chatbots:** Build conversational agents that can interact with users in a natural way.

**Key Features of LangChain:**

* **Modular Design:** LangChain is built around the concept of chains, which are sequences of components that work together to achieve a specific goal. Thi

In [ ]:
snapshot = app.get_state(config=thread_config)
dir(snapshot)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_make',
 '_replace',
 'config',
 'count',
 'created_at',
 'index',
 'metadata',
 'next',
 'parent_config',
 'tasks',
 'values']

In [ ]:
snapshot.values

{'messages': [HumanMessage(content='what is langchain', additional_kwargs={}, response_metadata={}, id='963bcd9b-a5b1-42a8-a2bd-9320a9d26805'),
  AIMessage(content="I can't provide information about LangChain using the available tools. I am only able to use the `default_api` which is limited in functionality. \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-9e9a6f1f-19b1-4f49-80a7-3ebe0e008e22-0', usage_metadata={'input_tokens': 78, 'output_tokens': 32, 'total_tokens': 110, 'input_token_details': {'cache_read':

In [ ]:
snapshot.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef9a9d4-f479-6759-8004-c22dd5589704'}}

In [ ]:
snapshot.metadata

{'source': 'loop',
 'writes': {'chat_bot': {'messages': [AIMessage(content='LangChain is a framework for developing applications powered by large language models (LLMs). It allows you to connect LLMs to various data sources and tools, enabling them to perform complex tasks like:\n\n* **Question Answering:** Retrieve information from databases, websites, or documents and answer questions based on the retrieved data.\n* **Summarization:** Summarize long documents or articles.\n* **Text Generation:** Generate creative content, write different kinds of text formats (emails, articles, code), and translate languages.\n* **Chatbots:** Build conversational agents that can interact with users in a natural way.\n\n**Key Features of LangChain:**\n\n* **Modular Design:** LangChain is built around the concept of chains, which are sequences of components that work together to achieve a specific goal. This modularity allows for easy customization and extension.\n* **Data Connectivity:** LangChain pro

In [ ]:
snapshot.next

()

In [ ]:
dir(app)

['InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abatch_with_config',
 '_abc_impl',
 '_acall_with_config',
 '_aprepare_state_snapshot',
 '_atransform_stream_with_config',
 '_batch_with_config',
 '_call_with_config',
 '_defaults',
 '_is_protocol',
 '_prepare_state_snapshot',
 '_transform_stream_with_config',
 'abatch',
 'abatch_as_completed',
 'aget_graph',
 'aget_state',
 'aget_state_history',
 'aget_subgraphs',
 'ainvoke',
 'as_tool',
 'assign',
 'astream',
 'astream_events',
 'astream_log',
 'atrans

In [ ]:
dir(graph_builder)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_schema',
 '_all_edges',
 'add_conditional_edges',
 'add_edge',
 'add_node',
 'branches',
 'channels',
 'compile',
 'compiled',
 'config_schema',
 'edges',
 'input',
 'managed',
 'nodes',
 'output',
 'schema',
 'schemas',
 'set_conditional_entry_point',
 'set_entry_point',
 'set_finish_point',
 'support_multiple_edges',
 'validate',
 'waiting_edges']